In [3]:
DRY_RUN = False
# Give students credit for completing per reviews
UPLOAD_SCORES_FOR_COMPLETION = True
completion_assignment_id = 8506682
POINTS_PER_REVIEW = 2
import requests
import pandas as pd
import json
import os
%load_ext dotenv
%dotenv

url = "https://bcourses.berkeley.edu/"
course_id = os.environ['COURSE_ID']
token = os.environ['TOKEN']
assignment_id = os.environ['ASSIGNMENT_ID']

def course_url(end=''):
    return f'{url}/api/v1/courses/{course_id}/{end}'

auth_token = {'Authorization': f'Bearer {token}'}

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [4]:
assignmentInfo = requests.get(
    course_url(f'assignments/{assignment_id}'),
    headers=auth_token
)

# Extracting assignment rubric id/rubric for the assignment
assignmentInfo = json.loads(assignmentInfo.text)

rubric_id = str(assignmentInfo['rubric_settings']['id'])

payload = {'include': 'peer_assessments', 'style': 'full'}
r = requests.get(course_url(f'/rubrics/{rubric_id}'),
                    params=payload,
                    headers=auth_token)


In [7]:
rubric_return = json.loads(r.text)

# Obtaining assessor_id (person who did peer review), score for the peer reviews
assessments_df = pd.DataFrame(rubric_return['assessments'])

# Obtaining user_id (person who was peer reviewed), completion and submission comments
peerReview = requests.get(course_url(f'/assignments/{assignment_id}/peer_reviews'),
                            headers=auth_token)

In [25]:
reviews = json.loads(peerReview.text)
incompletes = list(filter(lambda r: r['workflow_state'] != 'completed', reviews))
# len(incompletes)
incompletes

[{'id': 525219,
  'user_id': 5527180,
  'asset_id': 123434380,
  'asset_type': 'Submission',
  'workflow_state': 'assigned',
  'assessor_id': 5670378},
 {'id': 525277,
  'user_id': 5532003,
  'asset_id': 123434398,
  'asset_type': 'Submission',
  'workflow_state': 'assigned',
  'assessor_id': 5573933},
 {'id': 525305,
  'user_id': 5532039,
  'asset_id': 123434399,
  'asset_type': 'Submission',
  'workflow_state': 'assigned',
  'assessor_id': 5566821},
 {'id': 525185,
  'user_id': 5536298,
  'asset_id': 124167550,
  'asset_type': 'Submission',
  'workflow_state': 'assigned',
  'assessor_id': 5658264},
 {'id': 525052,
  'user_id': 5562686,
  'asset_id': 123434421,
  'asset_type': 'Submission',
  'workflow_state': 'assigned',
  'assessor_id': 5614235},
 {'id': 525276,
  'user_id': 5569833,
  'asset_id': 123434447,
  'asset_type': 'Submission',
  'workflow_state': 'assigned',
  'assessor_id': 5573933},
 {'id': 525304,
  'user_id': 5569904,
  'asset_id': 123434448,
  'asset_type': 'Submissi

In [8]:
peerReview_df = pd.read_json(peerReview.text)
peerReview_df['user_id'] = peerReview_df['user_id'].astype(str)

In [26]:
merged_df = pd.merge(peerReview_df, assessments_df, how='outer', left_on=[
                        'assessor_id', 'asset_id'], right_on=['assessor_id', 'artifact_id'])

In [51]:
def safe_len(x):
    try:
        return len(x)
    except:
        return -1
    
completed_reviews = merged_df[merged_df['workflow_state'] == 'completed']
merged_df['rubric_rows'] = merged_df['data'].apply(safe_len)
# data.where(data < 4)
merged_df[merged_df['rubric_rows'] != 4]


,id_x,user_id,asset_id,asset_type,workflow_state,assessor_id,id_y,rubric_id,rubric_association_id,score,artifact_id,artifact_type,assessment_type,artifact_attempt,data,rubric_association,rubric_rows
52,525219,5527180,123434380,Submission,assigned,5670378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
104,525277,5532003,123434398,Submission,assigned,5573933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
107,525305,5532039,123434399,Submission,assigned,5566821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
128,525185,5536298,124167550,Submission,assigned,5658264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
143,525052,5562686,123434421,Submission,assigned,5614235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
219,525276,5569833,123434447,Submission,assigned,5573933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
222,525304,5569904,123434448,Submission,assigned,5566821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
238,525221,5571096,123434454,Submission,assigned,5670378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
268,525303,5573724,123434470,Submission,assigned,5566821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
279,525597,5573996,123434475,Submission,assigned,5595987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1


In [9]:
peerReview_df

,id,user_id,asset_id,asset_type,workflow_state,assessor_id
0,525108,5443655,123434358,Submission,completed,5612108
1,525315,5443655,123434358,Submission,completed,5537816
2,525532,5443655,123434358,Submission,completed,5581108
3,524969,5523532,123434361,Submission,completed,5662511
4,525311,5523532,123434361,Submission,completed,5525946
...,...,...,...,...,...,...
712,525181,5677460,125212096,Submission,completed,5580524
713,525356,5677460,125212096,Submission,completed,5573553
714,525969,5678306,125011821,Submission,completed,5618047
715,525991,5678306,125011821,Submission,completed,5580877


In [ ]:
groups = merged_df.groupby('user_id')
scores = pd.DataFrame(groups['score'])
# get the 2 largest scores from the 2nd column of scores
scores['max'] = scores[1].apply(lambda x: sorted(x, reverse=True)[:-1])
scores['mean'] = scores['max'].apply(lambda x: round(sum(x) / len(x), 2))
scores['user_id'] = scores[0].astype(str)
